Importações

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
import re

Funções

In [ ]:
#Aplicar regex para transformar valores da coluna "Gasto"
def AlterarGastos(dataframe):
    return dataframe['Gasto'].apply(lambda x: float(re.sub(r'KWh$', '', x).replace(',', '.')))

Leitura de Arquivos e formatações

In [ ]:
# Lendo o arquivo CSV de treinamento e criando um DataFrame, e tratando
df_train = AlterarGastos(pd.read_csv('PresetArquivoTrain.csv', sep=';', header=0))

# Lendo o arquivo CSV de teste e criando um DataFrame, e tratando
df_test = AlterarGastos(pd.read_csv('PresetArquivoTeste.csv', sep=';', header=0))

Separando conjunto de dados

In [ ]:
# Separando as características (features) do seu conjunto de dados de treinamento
X_train = df_train[['Eletrodomestico', 'Comodo', 'Gasto']]

# Separando as características (features) do seu conjunto de dados de teste
X_test = df_test[['Eletrodomestico', 'Comodo', 'Gasto']]

Criando ordenador para codificar colunas não numericas

In [ ]:
# Definindo as colunas que serão codificadas
cat_cols = ['Eletrodomestico', 'Comodo']

# Criando o objeto OneHotEncoder
ohe = OneHotEncoder()

# Criando o objeto ColumnTransformer para aplicar o OneHotEncoder nas colunas desejadas
ct = ColumnTransformer([('encoder', ohe, cat_cols)], remainder='passthrough')

Aplicando codificação nas colunas

In [ ]:
# Aplicando o ColumnTransformer nos dados de treinamento
X_train = ct.fit_transform(X_train)

# Aplicando o ColumnTransformer nos dados de teste
X_test = ct.transform(X_test)

Criando IsolationForest

In [ ]:
# Criando um objeto Isolation Forest com os parâmetros padrão
clf = IsolationForest(random_state=0)

Treinando Modelo

In [ ]:
# Treinando o modelo com os dados de treinamento
clf.fit(X_train)

Realizando a previsão

In [ ]:
# Realizando a previsão (labels) do modelo no conjunto de dados de teste
y_pred = clf.predict(X_test)

Separando as anomalias

In [15]:
# Criando um DataFrame com as anomalias encontradas
df_anomaly = pd.DataFrame()
for i, y in enumerate(y_pred):
    if y == -1:
        df_row = df_test.iloc[[i]]
        if df_row['Eletrodomestico'].values[0] in df_train['Eletrodomestico'].unique():
            df_train_gasto = df_train[df_train['Eletrodomestico'] == df_row['Eletrodomestico'].values[0]]['Gasto']
            std_dev = df_train_gasto.std()
            mean = df_train_gasto.mean()
            if df_row['Gasto'].values[0] > (mean + 3*std_dev) or df_row['Gasto'].values[0] < (mean - 3*std_dev):
                df_anomaly = pd.concat([df_anomaly, df_row])

# Imprimindo as anomalias encontradas
print("Anomalias encontradas:")
print(df_anomaly)

Anomalias encontradas:
Empty DataFrame
Columns: []
Index: []
